# Worldwide Military Expenditure 2022

With data frim the [SIPRI Military Expenditure Database](https://www.sipri.org/databases/milex) for the #30DayChartChallenge 2024, day 1.

Data preparation for a voronoi treemap with the `d3.js` library.

In [ ]:
import json

import pandas as pd

In [ ]:
sipri_raw = pd.read_excel(
    "data/SIPRI-Milex-data-1949-2023.xlsx", sheet_name="Constant (2022) US$", skiprows=5,
)

In [ ]:
# Set "Country" as index
sipri = sipri_raw.set_index("Country")

# Remove "Notes" column
sipri = sipri.drop(columns="Notes")

# Convert column names to strings
sipri.columns = sipri.columns.astype(str)

# Set dytpes for columns to float replacing "..." and "xxx" with NaN
for col in sipri.columns:
    sipri[col] = pd.to_numeric(sipri[col], errors='coerce')

# Remove empty rows and columns (all NaN)
sipri = sipri.dropna(how='all')
sipri = sipri.dropna(axis=1, how='all')

# Set 'real' value since SIPRI data is in million
sipri = sipri * 1e6

# Reset index and order by "Country"
sipri = sipri.reset_index().sort_values("Country").reset_index(drop=True)

# Keep only data for 2022
sipri_2022 = sipri[["Country", "2022"]].copy().dropna().sort_values("2022", ascending=False)


In [ ]:
# Import NATO membership data from json to dict
with open("data/nato_memberships.json") as f:
    nato_memberships = json.load(f)

# Add NATO membership column
sipri_2022["NATO"] = sipri_2022["Country"].apply(lambda x: "NATO" if x in nato_memberships["2022"] else "None")

In [ ]:
sipri_voronoi = sipri_2022[["Country", "NATO", "2022"]].copy()

# Set column names
sipri_voronoi.columns = ["country", "group", "expenditure"]

sipri_voronoi["pct_of_group"] = sipri_voronoi.groupby("group")["expenditure"].transform(
    lambda x: x / x.sum()
)

sipri_voronoi["pct_of_total"] = sipri_voronoi["expenditure"] / sipri_voronoi["expenditure"].sum()

# Order by expenditure
sipri_voronoi = sipri_voronoi.sort_values("expenditure", ascending=False)

# Rename some countries
countries_rename = {
    "United States of America": "USA",
    "Korea, South": "South Korea",
    "United Kingdom": "UK",
}
sipri_voronoi["country"] = sipri_voronoi["country"].replace(countries_rename)

# Export to json
sipri_voronoi.to_json("voronoi/src/data/sipri_voronoi.json", orient="records")

sipri_voronoi
